# Python 赋值机制

先看一个例子：

In [2]:
x = [1, 2, 3]
y = x
x[1] = 100
print(y)

[1, 100, 3]


改变变量`x`的值，变量`y`的值也随着改变，这与**Python**内部的赋值机制有关。

但是，请注意，如果我们采取的是切片而不是赋值，则 x 的改变不影响 y

In [3]:
x = [1, 2, 3]
z = x[:]
x[1] = 100
print(z)

[1, 2, 3]


## 简单类型

先来看这一段代码在**Python**中的执行过程。

```python
x = 500
y = x
y = 'foo'
```

1. `x = 500`

**Python**分配了一个 `PyInt` 大小的内存 `pos1` 用来储存对象 `500` ，然后，Python在命名空间中让变量 `x` 指向了这一块内存，注意，整数是不可变类型，所以这块内存的内容是不可变的。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变)|          `x : pos1`         |


2. `y = x `

**Python**并没有使用新的内存来储存变量 `y` 的值，而是在命名空间中，让变量 `y` 与变量 `x` 指向了同一块内存空间。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变)|`x : pos1`<br> `y : pos1`|

3. `y = 'foo'`

**Python**此时分配一个 `PyStr` 大小的内存 `pos2` 来储存对象 `foo` ，然后改变变量 `y` 所指的对象。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变)<br> `pos2 : PyStr('foo')` (不可变)|`x : pos1`<br>`y : pos2`|

对这一过程进行验证，可以使用 `id` 函数。

    id(x)

返回变量 `x` 的内存地址。

In [2]:
x = 500
y = x
print('\n',id(x),'\n',id(y),'\n',x is y)


 2308538774064 
 2308538774064 
 True


现在 `y` 指向另一块内存：

In [3]:
y = 'foo'
print('\n',id(y),'\n',x is y)


 2308506354832 
 False


**Python**会为每个出现的对象进行赋值，哪怕它们的值是一样的，例如：

In [5]:
x = 500
y = 500
print('\n',id(x),'\n',id(y),'\n',x is y)


 2308538773744 
 2308538773872 
 False


不过，为了提高内存利用效率，对256的以下整数和短字符串对象进行复用，**Python**采用了重用对象内存的办法：

In [48]:
x = 256
y = 256
x is y

True

In [49]:
x = 257
y = 257
x is y

False

## 容器类型

### List  赋值与切片赋值

看下面三段代码：

1. 生成原始列表；

2. 对`list`的赋值与切片赋值；

3. 分别改变赋值和切片赋值后的对象；

4. 重新生成 `y`。

``` python
x = [500, 501, 502]
```

``` python
y = x
z = x[:]
```

``` python
y[1] = 600
z[1] = 1000
```

``` python
y = [700,800]
```

1. `x = [500, 501, 502]`

Python为3个PyInt分配内存 `pos1` ， `pos2` ， `pos3` （不可变），然后为列表分配一段内存 `pos10` ，它包含3个位置，分别指向这3个内存，最后再让变量 `x` 指向这个列表。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变) <br> `pos2 : PyInt(501)` (不可变) <br>`pos3 : PyInt(502)` (不可变) <br> `pos10 : PyList(pos1, pos2, pos3)` (可变)|`x : pos10`|


In [22]:
x = [500, 501, 502]
print('\n',id(x[0]),
      '\n',id(x[1]),
      '\n',id(x[2]),
      '\n',id(x))


 1744431031440 
 1744431031120 
 1744431031152 
 1744430279240


2. `y = x;  z=x[:]`

`y = x`并没有创建新的对象，只需要将 `y` 指向 `pos10` 即可，`z = x[:]`创建了新的对象`pos11`。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变) <br> `pos2 : PyInt(501)` (不可变) <br> `pos3 : PyInt(502)` (不可变) <br> `pos10 : PyList(pos1, pos2, pos3)` (可变)<br> `pos11 : PyList(pos1, pos2, pos3)` (可变)|`x : pos10`<br>`y : pos10`<br>`z : pos11`|



下面我们来验证：

赋值，`id(y)` 与 `id(x)` 相同，`id(z)` 与 `id(x)` 不同。

In [23]:
y = x
z = x[:]
print('\n id(x):',id(x),
      '\n id(y):',id(y),
      '\n id(z):',id(z),
      '\n x is y:',x is y,
      '\n x is z:',x is z)


 id(x): 1744430279240 
 id(y): 1744430279240 
 id(z): 1744430280456 
 x is y: True 
 x is z: False


x 与 z 的元素对应的指针是一样的,因为x与 z 的元素是相同的：

In [24]:
for i in range(3):
    print( id(x[i]),id(z[i]))

1744431031440 1744431031440
1744431031120 1744431031120
1744431031152 1744431031152


3. `y[1] = 600;  z[1]=1000`

`y[1] = 600`:

- 原来 `y[1]` 这个位置指向的是 `pos2` ，由于不能修改 `pos2` 的值，所以首先为 `600` 分配新内存 `pos5` 。

- 再把 `y[1]` 指向的位置修改为 `pos5` 。此时，由于 `pos2` 位置的对象已经没有用了，**Python**会自动调用垃圾处理机制将它回收。

`z[1]=1000`:

- 首先为 `1000` 分配新内存 `pos6`;

- 再把 `z[1]` 指向的位置修改为 `pos6`。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变) <br> `pos2 :` 垃圾回收 <br> `pos3 : PyInt(502)` (不可变)<br>`pos5 : PyInt(600)` (不可变) <br> `pos6 : PyInt(1000)` (不可变)<br> `pos10 : PyList(pos1, pos5, pos3)` (可变)<br> `pos11 : PyList(pos1, pos6, pos3)` (可变) |`x : pos10`<br> `y : pos10`<br> `z : pos11`|



我们来验证下修改 `y[1]/z[1]` 后：

* `id(y)/id(z)/id(x)` 不改变；

* `id(x[1])` , `id(y[1])`，`id(z[1])` 的值改变了。

In [27]:
print('\n 修改前的id(y):',id(y),
      '\n 修改前的id(y):',id(x),
    '\n 修改前的id(z):',id(z),
    '\n 修改前的id(y[1]):',id(y[1]),
    '\n 修改前的id(x[1]):',id(x[1]),
    '\n 修改前的id(z[1]):',id(z[1]),
    '\n 修改前id(y[1]) 是否等于 id(x[1]):',id(y[1]) == id(x[1]),
    '\n 修改前id(z[1]) 是否等于 id(x[1]):',id(z[1]) == id(x[1]))
y[1] = 600
z[1] = 1000
print('\n 修改后的id(y):',id(y),
      '\n 修改前的id(y):',id(x),
    '\n 修改后的id(z):',id(z),
    '\n 修改后的id(y[1]):',id(y[1]),
    '\n 修改后的id(x[1]):',id(x[1]),
    '\n 修改后的id(z[1]):',id(z[1]),
    '\n 修改后id(y[1]) 是否等于 id(x[1]):',id(y[1]) == id(x[1]),
    '\n修改后id(z[1]) 是否等于 id(x[1]):',id(z[1]) == id(x[1]))


 修改前的id(y): 1744430279240 
 修改前的id(y): 1744430279240 
 修改前的id(z): 1744430280456 
 修改前的id(y[1]): 1744431031728 
 修改前的id(x[1]): 1744431031728 
 修改前的id(z[1]): 1744431031344 
 修改前id(y[1]) 是否等于 id(x[1]): True 
 修改前id(z[1]) 是否等于 id(x[1]): False

 修改后的id(y): 1744430279240 
 修改前的id(y): 1744430279240 
 修改后的id(z): 1744430280456 
 修改后的id(y[1]): 1744431031824 
 修改后的id(x[1]): 1744431031824 
 修改后的id(z[1]): 1744431031568 
 修改后id(y[1]) 是否等于 id(x[1]): True 
修改后id(z[1]) 是否等于 id(x[1]): False


4. `y = [700, 800]`

首先创建这个列表，然后将变量 `y` 指向它。

|内存|命名空间|
|---|---|
|`pos1 : PyInt(500)` (不可变) <br> `pos2 :` 垃圾回收 <br> `pos3 : PyInt(502)` (不可变)<br>`pos5 : PyInt(600)` (不可变) <br> `pos6 : PyInt(1000)` (不可变)<br>`pos7 : PyInt(700)` (不可变)<br>`pos8 : PyInt(800)` (可变)<br> `pos10 : PyList(pos1, pos5, pos3)` (可变)<br> `pos11 : PyList(pos1, pos6, pos3)` (可变)<br> `pos12 : PyList(pos7, pos8)` (可变)|`x : pos10`<br> `z : pos11`<br> `y : pos12`|



对这一过程进行验证：

更改 `y` 的值，`id(y)` 的值改变

In [28]:
y = [700, 800]
print('\n',id(y),
      '\n',id(x))


 1744431198536 
 1744430279240


# 判断语句

## 基本用法

判断，基于一定的条件，决定是否要执行特定的一段代码。

一个完整的 `if` 结构通常如下所示（注意：条件后的 `:` 是必须要的，缩进值需要一样）：
    
    if <condition 1>:
        <statement 1>
        <statement 2>
    elif <condition 2>: 
        <statements>
    else:
        <statements>

当条件1被满足时，执行 `if` 下面的语句，当条件1不满足的时候，转到 `elif` ，看它的条件2满不满足，满足执行 `elif` 下面的语句，不满足则执行 `else` 下面的语句。

In [2]:
x = 0
if x > 0:
    print("x is positive")
elif x == 0:
    print("x is zero")
else:
    print("x is negative")

x is zero


`elif` 的个数没有限制，可以是1个或者多个，也可以没有。

`else` 最多只有1个，也可以没有。

可以使用 `and` ， `or` , `not` 等逻辑关键词结合多个判断条件：

In [6]:
x = 10
y = -5
print('\n',x > 0 and y < 0,
      '\n',not x > 0,
      '\n',x < 0 or y < 0)


 True 
 False 
 True


##  缩进与代码块

In [8]:
x = 0.5
if x > 0:
    print("Hey!")
    print("x is positive")

Hey!
x is positive


在这里，如果 `x > 0` 为 `False` ，那么程序将不会执行两条 `print` 语句。

虽然都是用 `if` 关键词定义判断，但与**C，Java**等语言不同，**Python**不使用 `{}` 将 `if` 语句控制的区域包含起来。**Python**使用的是缩进方法。同时，也不需要用 `()` 将判断条件括起来。

上面例子中的这两条语句：
```python
    print("Hey!")
    print("x is positive")
```
就叫做一个**代码块**，同一个代码块使用同样的缩进值，它们组成了这条 `if` 语句的主体。

不同的缩进值表示不同的代码块，例如：

`x < 0` 时：

In [10]:
x = -0.5
if x > 0:
    print("Hey!")
    print("x is positive")
    print("This is still part of the block")
print("This isn't part of the block, and will always print.")

This isn't part of the block, and will always print.


## 值作为判断条件

**Python**不仅仅可以使用布尔型变量作为条件，它可以直接在`if`中使用任何表达式作为条件：

大部分表达式的值都会被当作`True`，但以下表达式值会被当作`False`：

- False
- None
- 0
- 空字符串，空列表，空字典，空集合

In [11]:
mylist = [3, 1, 4, 1, 5, 9]
if mylist:
    print("The first element is:", mylist[0])
else:
    print("There is no first element.")

The first element is: 3


修改为空列表：

In [12]:
mylist = []
if mylist:
    print("The first element is:", mylist[0])
else:
    print("There is no first element.")

There is no first element.


当然这种用法并不推荐，推荐使用 `if len(mylist) > 0:` 来判断一个列表是否为空。

# 循环

循环的作用在于将一段**代码块**重复执行多次。

## while循环

    while <condition>:
        <statesments>

**Python**会循环执行`<statesments>`，直到`<condition>`不满足为止。

例如，计算数字`0`到`1000000`的和：

In [17]:
i = 0
total = 0
while i<100000:
    total += i
    i += 1
print('\n',total,'\n',i)


 4999950000 
 100000


之前提到，空容器会被当成 `False` ，因此可以用 `while` 循环来读取容器中的所有元素：

In [18]:
plays = set(['Hamlet', 'Macbeth', 'King Lear'])
while plays:
    play = plays.pop()
    print('Perform', play)

Perform King Lear
Perform Macbeth
Perform Hamlet


循环每次从 `plays` 中弹出一个元素，一直到 `plays` 为空为止。

## for 循环

    for <variable> in <sequence>:
        <indented block of code>

`for` 循环会遍历完`<sequence>`中所有元素为止

上一个例子可以改写成如下形式：

In [19]:
plays = set(['Hamlet', 'Macbeth', 'King Lear'])
for play in plays:
    print('Perform', play)

Perform King Lear
Perform Macbeth
Perform Hamlet


使用 `for` 循环时，注意尽量不要改变 `plays` 的值，否则可能会产生意想不到的结果。

之前的求和也可以通过 `for` 循环来实现：

In [22]:
total = 0
for i in range(100000):
    total += i
print(total)

4999950000


In [23]:
type(range(10))

range

在 `python3` 中，`range(10)` 并不是在`python2`中那样，返回的是一个列表，而是一个`range`对象，所以在`python3`中不区分 `range` 和 `xrange`,只有 `range`。

## continue 语句

遇到 `continue` 的时候，程序会返回到循环的最开始重新执行。

例如在循环中忽略一些特定的值：

In [24]:
values = [7, 6, 4, 7, 19, 2, 1]
for i in values:
    if i % 2 != 0:
        # 忽略奇数
        continue
    print(i)

6
4
2


## break 语句

遇到 `break` 的时候，程序会跳出循环，不管下一个元素是否满足循环条件：

In [25]:
command_list = ['start', 
                'process', 
                'process',
                'process', 
                'stop', 
                'start', 
                'process', 
                'stop']
while command_list:
    command = command_list.pop(0)
    if command == 'stop':
        break
    print(command)

start
process
process
process


在遇到第一个 `'stop'` 之后，程序跳出循环。

## else语句

与 `if` 一样， `while` 和 `for` 循环后面也可以跟着 `else` 语句，不过要和`break`一起连用。

- 当循环未中途跳出时， `else` 被执行；否则 `else` 不执行。

中途跳出，不执行`else`：

In [27]:
values = [7, 6, 4, 7, 19, 2, 1]
for x in values:
    if x >= 10:
        print('Found:', x)
        break
else:
    print('All values greater than 10')

Found: 19


未中途跳出，循环结束，执行`else`：

In [28]:
values = [11, 12, 13, 100]
for x in values:
    if x <= 10:
        print('Found:', x)
        break
else:
    print('All values greater than 10')

All values greater than 10


# 列表推导式

针对集合、元组、字典、列表等，通过循环可以生成，但也可以通过列表推导式生成。

In [29]:
# for循环
values = [10, 21, 4, 7, 12]
squares = []
for x in values:
    squares.append(x**2)
print(squares)

[100, 441, 16, 49, 144]


列表推导式可以使用更简单的方法来创建这个列表：

In [30]:
values = [10, 21, 4, 7, 12]
squares = [x**2 for x in values]
print(squares)

[100, 441, 16, 49, 144]


还可以在列表推导式中加入条件进行筛选。例如在上面的例子中，假如只想保留列表中不大于`10`的数的平方：

In [32]:
values = [10, 21, 4, 7, 12]
squares = [x**2 for x in values if x<=10]
print(squares)

[100, 16, 49]


也可以使用推导式生成集合和字典：

In [33]:
square_set = {x**2 for x in values if x <= 10}
print(square_set)
square_dict = {x: x**2 for x in values if x <= 10}
print(square_dict)

{16, 49, 100}
{10: 100, 4: 16, 7: 49}


再如，计算上面例子中生成的列表中所有元素的和：

In [34]:
total = sum([x**2 for x in values if x <= 10])
print(total)

165


但是，**Python**会生成这个列表，然后在将它放到垃圾回收机制中（因为没有变量指向它），这毫无疑问是种浪费。

为了解决这种问题，与xrange()类似，**Python**使用产生式表达式来解决这个问题：

In [35]:
total = sum(x**2 for x in values if x <= 10)
print(total)

165


与上面相比，只是去掉了括号，但这里并不会一次性的生成这个列表。

# 函数

## 定义函数

函数`function`，通常接受输入参数，并有返回值。

它负责完成某项特定任务，而且相较于其他代码，具备相对的独立性。

In [36]:
def add(x, y):
    """Add two numbers"""
    a = x + y
    return a

函数通常有一下几个特征：
- 使用 `def` 关键词来定义一个函数。
-  `def` 后面是函数的名称，括号中是函数的参数，不同的参数用 `,` 隔开， `def foo():` 的形式是必须要有的，参数可以为空；
- 使用缩进来划分函数的内容；
-  `docstring` 用 `"""` 包含的字符串，用来解释函数的用途，可省略；
-  `return` 返回特定的值，如果省略，返回 `None` 。

## 使用函数

使用函数时，只需要将参数换成特定的值传给函数。

**Python**并没有限定参数的类型，因此可以使用不同的参数类型：

In [37]:
print(add(2, 3))
print(add('foo', 'bar'))

5
foobar


在这个例子中,

* 如果传入的两个参数不可以相加;
* 如果传入的参数数目与实际不符合：

那么**Python**会将报错。

传入参数时，Python提供了两种选项：

- 第一种是上面使用的按照位置传入参数；
- 另一种则是使用关键词模式，显式地指定参数的值：
- 混合使用两者。

In [39]:
print(add(x=2, y=3))
print(add(2, y=3))

5
5


## 设定参数默认值

可以在函数定义的时候给参数设定默认值，例如：

In [40]:
def quad(x, a=1, b=0, c=0):
    return a*x**2 + b*x + c

可以省略有默认值的参数：

In [41]:
print(quad(2.0))

4.0


可以修改参数的默认值：

In [43]:
print(quad(2.0, 2, c=4))

12.0


这里混合了位置和指定两种参数传入方式，第二个2是传给 `a` 的。

注意，在使用混合语法时，要注意不能给同一个值赋值多次，否则会报错，例如：
```python
print(quad(2.0, 2, a=2))
```

## 接受不定参数

使用如下方法，可以使函数接受不定数目的参数：

In [44]:
def add(x, *args):
    total = x
    for arg in args:
        total += arg
    return total

这里，`*args` 表示参数数目不定，可以看成一个**元组**，把第一个参数后面的参数当作元组中的元素。

In [45]:
print(add(1, 2, 3, 4))

10


这样定义的函数不能使用关键词传入参数，要使用关键词，可以这样：

In [47]:
def add(x, **kwargs):
    total = x
    for arg, value in kwargs.items():
        print("adding ", arg)
        total += value
    return total

这里， `**kwargs` 表示参数数目不定，相当于一个字典，关键词和值对应于键值对。

In [48]:
print(add(10, y=11, z=12, w=13))

adding  y
adding  z
adding  w
46


再看这个例子，可以接收任意数目的位置参数和键值对参数：

In [49]:
def foo(*args, **kwargs):
    print(args, kwargs)

foo(2, 3, x='bar', z=10)

(2, 3) {'x': 'bar', 'z': 10}


不过要按顺序传入参数，先传入位置参数 `args` ，在传入关键词参数 `kwargs` 。

## 返回多个值

函数可以返回多个值：

In [50]:
from math import atan2

def to_polar(x, y):
    r = (x**2 + y**2) ** 0.5
    theta = atan2(y, x)
    return r, theta

r, theta = to_polar(3, 4)
print(r, theta)

5.0 0.9272952180016122


事实上，**Python**将返回的两个值变成了元组：

In [51]:
print(to_polar(3,4))

(5.0, 0.9272952180016122)


因为这个元组中有两个值，所以可以使用

    r, theta = to_polar(3, 4)

给两个值赋值。

列表也有相似的功能：

In [52]:
a, b, c = [1, 2, 3]
print(a, b, c)

1 2 3


事实上，不仅仅返回值可以用元组表示，也可以将参数用元组以这种方式传入：

In [53]:
def add(x, y):
    """Add two numbers"""
    a = x + y
    return a
    
z = (2, 3)
print(add(*z))

z1 = {'x':2,'y':3}
print(add(**z1))

5
5


## map 方法生成序列

可以通过 `map` 的方式利用函数来生成序列：

In [55]:
def sqr(x): 
    return x ** 2

a = [2,3,4]
print(list(map(sqr, a)))

[4, 9, 16]


其用法为：
    
    map(aFun, aSeq)

将函数 `aFun` 应用到序列 `aSeq` 上的每一个元素上，返回一个列表，不管这个序列原来是什么类型。

事实上，根据函数参数的多少，`map` 可以接受多组序列，将其对应的元素作为参数传入函数：

In [57]:
def add(x, y): 
    return x + y

a = (2,3,4)
b = [10,5,3]
print(list(map(add,a,b)))

[12, 8, 7]


# 模块和包

## 模块

Python会将所有 `.py` 结尾的文件认定为Python代码文件，考虑下面的脚本 `ex1.py` ：

In [60]:
%%writefile ex1.py

PI = 3.1416

def sum(lst):
    tot = lst[0]
    for value in lst[1:]:
        tot = tot + value
    return tot
    
w = [0, 1, 2, 3]
print(sum(w), PI)

Overwriting ex1.py


可以执行它：

In [61]:
%run ex1.py

6 3.1416


这个脚本可以当作一个模块，可以使用`import`关键词加载并执行它（这里要求`ex1.py`在当前工作目录）：

In [62]:
import ex1

6 3.1416


In [63]:
ex1

<module 'ex1' from 'E:\\chen_gitlub\\python_notes\\ex1.py'>

在导入时，**Python**会执行一遍模块中的所有内容。

`ex1.py` 中所有的变量都被载入了当前环境中，不过要使用

    ex1.变量名

的方法来查看或者修改这些变量：

In [64]:
print(ex1.PI)

3.1416


In [65]:
ex1.PI = 3.141592653
print(ex1.PI)

3.141592653


还可以用

    ex1.函数名

调用模块里面的函数：

In [66]:
print(ex1.sum([2, 3, 4]))

9


删除之前生成的文件：

In [69]:
import os
os.remove('ex1.py')

## `__name__` 属性

有时候我们想将一个 `.py` 文件既当作脚本，又能当作模块用，这个时候可以使用 `__name__` 这个属性。

只有当文件被当作脚本执行的时候， `__name__`的值才会是 `'__main__'`，所以我们可以：

In [72]:
%%writefile ex2.py

PI = 3.1416

def sum(lst):
    """ Sum the values in a list
    """
    tot = 0
    for value in lst:
        tot = tot + value
    return tot

def add(x, y):
    " Add two values."
    a = x + y
    return a

def test():
    w = [0,1,2,3]
    assert(sum(w) == 6)
    print('test passed.')
    
if __name__ == '__main__':
    test()

Overwriting ex2.py


运行文件

In [73]:
%run ex2.py

test passed.


当作模块导入， `test()` 不会执行：

In [75]:
import ex2

但是可以使用其中的变量：

In [76]:
ex2.PI

3.1416

使用别名

In [77]:
import ex2 as e2
e2.PI

3.1416

## 其他导入方法

可以从模块中导入变量：

In [78]:
from ex2 import add, PI

使用 `from` 后，可以直接使用 `add` ， `PI`：

In [80]:
add(2, 3)

5

或者使用 `*` 导入所有变量：

In [81]:
from ex2 import *
add(3, 4.5)

7.5

这种导入方法不是很提倡，因为如果你不确定导入的都有哪些，可能覆盖一些已有的函数。

删除文件：

In [82]:
import os
os.remove('ex2.py')

## 包

假设我们有这样的一个文件夹：

foo/
- `__init__.py` 
- `bar.py` (defines func)
- `baz.py` (defines zap)

这意味着 foo 是一个包，我们可以这样导入其中的内容：

```python    
from foo.bar import func
from foo.baz import zap
```

`bar` 和 `baz` 都是 `foo` 文件夹下的 `.py` 文件。

导入包要求：
- 文件夹 `foo` 在**Python**的搜索路径中
- `__init__.py` 表示 `foo` 是一个包，它可以是个空文件。

## 常用的标准库

- re 正则表达式
- copy 复制
- math, cmath 数学
- decimal, fraction
- sqlite3 数据库
- os, os.path 文件系统
- gzip, bz2, zipfile, tarfile 压缩文件
- csv, netrc 各种文件格式
- xml
- htmllib
- ftplib, socket
- cmd 命令行
- pdb 
- profile, cProfile, timeit
- collections, heapq, bisect 数据结构
- mmap
- threading, Queue 并行
- multiprocessing
- subprocess
- pickle, cPickle
- struct

## PYTHONPATH设置

Python的搜索路径可以通过环境变量PYTHONPATH设置，环境变量的设置方法依操作系统的不同而不同，具体方法可以网上搜索。

# 异常

## try & except 块

写代码的时候，出现错误必不可免，即使代码没有问题，也可能遇到别的问题。

看下面这段代码：

```python 
import math

while True:
    text = raw_input('> ')
    if text[0] == 'q':
        break
    x = float(text)
    y = math.log10(x)
    print "log10({0}) = {1}".format(x, y)
```

这段代码接收命令行的输入，当输入为数字时，计算它的对数并输出，直到输入值为 `q` 为止。

乍看没什么问题，然而当我们输入0或者负数时：

In [86]:
import math

while True:
    text = input('> ')
    if text[0] == 'q':
        break
    x = float(text)
    y = math.log10(x)
    print("log10({0}) = {1}".format(x, y))

> -1


ValueError: math domain error

`log10` 函数会报错，因为不能接受非正值。

一旦报错，程序就会停止执行，如果不希望程序停止执行，那么我们可以添加一对 `try & except`： 

```python
import math

while True:
    try:
        text = raw_input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = math.log10(x)
        print "log10({0}) = {1}".format(x, y)
    except ValueError:
        print "the value must be greater than 0"
```

一旦 `try` 块中的内容出现了异常，那么 `try` 块后面的内容会被忽略，**Python**会寻找 `except` 里面有没有对应的内容，如果找到，就执行对应的块，没有则抛出这个异常。

在上面的例子中，`try` 抛出的是 `ValueError`，`except` 中有对应的内容，所以这个异常被 `except` 捕捉到，程序可以继续执行：

In [89]:
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = math.log10(x)
        print("log10({0}) = {1}".format(x, y))
    except ValueError:
        print("the value must be greater than 0")

> -1
the value must be greater than 0
> 0
the value must be greater than 0
> 1
log10(1.0) = 0.0
> q


## 捕捉不同的错误类型

``` python
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = 1 / math.log10(x)
        print("log10({0}) = {1}".format(x, y))
    except ValueError:
        print("the value must be greater than 0")
```

假设我们将这里的 `y` 更改为 `1 / math.log10(x)`，此时输入 `1`：

In [92]:
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = 1 / math.log10(x)
        print("log10({0}) = {1}".format(x, y))
    except ValueError:
        print("the value must be greater than 0")

> 1


ZeroDivisionError: float division by zero

因为我们的 `except` 里面并没有 `ZeroDivisionError`，所以会抛出这个异常，我们可以通过两种方式解决这个问题：

### 捕捉所有异常

将`except` 的值改成 `Exception` 类，来捕获所有的异常。

In [93]:
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = 1 / math.log10(x)
        print("log10({0}) = {1}".format(x, y))
    except Exception:
        print("invalid value")

> 1
invalid value
> 0
invalid value
> 2
log10(2.0) = 3.321928094887362
> q


### 指定特定值

这里，我们把 `ZeroDivisionError` 加入 `except` 。

In [95]:
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = 1 / math.log10(x)
        print("1 / log10({0}) = {1}".format(x, y))
    except (ValueError, ZeroDivisionError):
        print("invalid value")

> 1
invalid value
> 2
1 / log10(2.0) = 3.321928094887362
> q


或者另加处理：

In [97]:
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = 1 / math.log10(x)
        print("1 / log10({0}) = {1}".format(x, y))
    except ValueError:
        print("the value must be greater than 0")
    except ZeroDivisionError:
        print("the value must not be 1")

> 1
the value must not be 1
> 0
the value must be greater than 0
> 2
1 / log10(2.0) = 3.321928094887362
> q


事实上,我们还可以将这两种方式结合起来,用 `Exception` 来捕捉其他的错误：

In [98]:
import math

while True:
    try:
        text = input('> ')
        if text[0] == 'q':
            break
        x = float(text)
        y = 1 / math.log10(x)
        print("1 / log10({0}) = {1}".format(x, y))
    except ValueError:
        print("the value must be greater than 0")
    except ZeroDivisionError:
        print("the value must not be 1")
    except Exception:
        print("unexpected error")

> 0
the value must be greater than 0
> 1
the value must not be 1
> 2
1 / log10(2.0) = 3.321928094887362
> q


## 自定义异常

异常是标准库中的类，这意味着我们可以自定义异常类：

In [104]:
class CommandError(ValueError):
    pass

这里我们定义了一个继承自 `ValueError` 的异常类，异常类一般接收一个字符串作为输入，并把这个字符串当作异常信息，例如：

In [105]:
valid_commands = {'start', 'stop', 'pause'}

while True:
    command = input('> ')
    if command.lower() not in valid_commands:
        raise CommandError('Invalid commmand: %s' % command)

> bad


CommandError: Invalid commmand: bad

我们使用 `raise` 关键词来抛出异常。

我们可以使用 `try/except` 块来捕捉这个异常：

``` python
valid_commands = {'start', 'stop', 'pause'}

while True:
    command = input('> ')
    try:
        if command.lower() not in valid_commands:
            raise CommandError('Invalid commmand: %s' % command)
    except CommandError:
        print('Bad command string: "%s"' % command)
```

由于 `CommandError` 继承自 `ValueError`，我们也可以使用 `except ValueError` 来捕获这个异常。

## finally

try/catch 块还有一个可选的关键词 finally。

不管 try 块有没有异常， finally 块的内容总是会被执行，而且会在抛出异常前执行，因此可以用来作为安全保证，比如确保打开的文件被关闭。。

In [107]:
try:
    print(1)
finally:
    print('finally was called.') 

1
finally was called.


在抛出异常前执行：

In [108]:
try:
    print(1 / 0) 
finally:
    print('finally was called.')

finally was called.


ZeroDivisionError: division by zero

如果异常被捕获了，在最后执行：

In [109]:
try:
    print(1 / 0)
except ZeroDivisionError:
    print('divide by 0.')
finally:
    print('finally was called.')

divide by 0.
finally was called.


# 异常

出现了一些需要让用户知道的问题，但又不想停止程序，这时候我们可以使用警告：

首先导入警告模块：

In [1]:
import warnings

在需要的地方，我们使用 `warnings` 中的 `warn` 函数：

    warn(msg, WarningType = UserWarning)

In [3]:
def month_warning(m):
    if not 1<= m <= 12:
        msg = "month (%d) is not between 1 and 12" % m
        warnings.warn(msg, RuntimeWarning)

month_warning(13)

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: month (13) is not between 1 and 12
  after removing the cwd from sys.path.


有时候我们想要忽略特定类型的警告，可以使用 `warnings` 的 `filterwarnings` 函数：

    filterwarnings(action, category)

将 `action` 设置为 `'ignore'` 便可以忽略特定类型的警告：

In [5]:
warnings.filterwarnings(action = 'ignore', category = RuntimeWarning)

month_warning(13)

# 文件读写

写入测试文件：

In [18]:
%%writefile test.txt
this is a test file.
hello world!
python is good!
today is a good day.

Overwriting test.txt


## 读文件

使用 `open` 函数或者 `file` 函数来读文件，使用文件名的字符串作为输入参数：

In [19]:
f = open('test.txt')

默认以读的方式打开文件，如果文件不存在会报错。
可以使用 `read` 方法来读入文件中的所有内容：

In [20]:
text = f.read()
print(text)
f.close()

this is a test file.
hello world!
python is good!
today is a good day.



也可以按照行读入内容，`readlines` 方法返回一个列表，每个元素代表文件中每一行的内容：

In [21]:
f = open('test.txt')
lines = f.readlines()
print(lines)
# 使用完文件之后，需要将文件关闭。
f.close()

['this is a test file.\n', 'hello world!\n', 'python is good!\n', 'today is a good day.\n']


事实上，我们可以将 `f` 放在一个循环中，得到它每一行的内容：

In [22]:
f = open('test.txt')
for line in f:
    print(line)
f.close()

this is a test file.

hello world!

python is good!

today is a good day.



In [1]:
# 删除刚刚创建的文件
import os
os.remove('test.txt')

## 写文件

我们使用 `open` 函数的写入模式来写文件：

In [2]:
f = open('myfile.txt', 'w')
f.write('hello world!')
f.close()

使用 `w` 模式时，如果文件不存在会被创建，我们可以查看是否真的写入成功：

In [3]:
print(open('myfile.txt').read())

hello world!


如果文件已经存在， `w` 模式会覆盖之前写的所有内容：

In [4]:
f = open('myfile.txt','w')
f.write('another hello world!')
f.close()
print(open('myfile.txt').read())

another hello world!


除了写入模式，还有追加模式 `a` ，追加模式不会覆盖之前已经写入的内容，而是在之后继续写入：

In [6]:
f = open('myfile.txt', 'a')
f.write('... and more')
f.close()
print(open('myfile.txt').read())

another hello world!... and more


写入结束之后一定要将文件关闭，否则可能出现内容没有完全写入文件中的情况。

还可以使用读写模式 `w+`：

In [7]:
f = open('myfile.txt', 'w+')
f.write('hello world!')
f.seek(6)
print(f.read())
f.close()

world!


这里 `f.seek(6)` 移动到文件的第6个字符处，然后 `f.read()` 读出剩下的内容。

In [8]:
import os
os.remove('myfile.txt')

## 二进制文件

二进制读写模式 b：

In [9]:
import os
f = open('binary.bin', 'wb')
f.write(os.urandom(16))
f.close()

f = open('binary.bin', 'rb')
print(repr(f.read()))
f.close()

b'\xb9\xdc*\xcf\x04^\xd0F\xf6\xd9|y0N\xaf\x0e'


In [10]:
import os
os.remove('binary.bin')

## 换行符

不同操作系统的换行符可能不同：

- `\r`
- `\n`
- `\r\n`

使用 `U` 选项，可以将这三个统一看成 `\n` 换行符。

## 关闭文件

在**Python**中，如果一个打开的文件不再被其他变量引用时，它会自动关闭这个文件。

所以正常情况下，如果一个文件正常被关闭了，忘记调用文件的 `close` 方法不会有什么问题。

关闭文件可以保证内容已经被写入文件，而不关闭可能会出现意想不到的结果：

In [11]:
f = open('newfile.txt','w')
f.write('hello world')
g = open('newfile.txt', 'r')
print(repr(g.read()))

''


虽然这里写了内容，但是在关闭之前，这个内容并没有被写入磁盘。

使用循环写入的内容也并不完整：

In [14]:
f.close()
g.close()
import os
os.remove('newfile.txt')

## with 方法

事实上，**Python**提供了更安全的方法，当 `with` 块的内容结束后，**Python**会自动调用它的`close` 方法，确保读写的安全：

与 `try/exception/finally` 效果相同，但更简单。

In [15]:
with open('newfile.txt','w') as f:
    f.write('hello world: ')

In [17]:
g = open('newfile.txt', 'r')
print(g.read())
g.close()

hello world: 
